# Assignment 3
Training a simple neural named entity recognizer (NER)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/


The three files (train, test and eval) are available from the course git repository (https://github.com/kfirbar/nlp-course)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion *read_data* for reading the data from a single file (either train, test or eval). This function recieves a filepath and returns a list of sentence. Every sentence is encoded as a pair of lists, one list contains the words and one list contains the labels.

In [ ]:
def read_data(filepath):
    data = []
    f = open(filepath, "r")
    lines = f.readlines()
    words = []
    labels = []
    for line in lines:
      line_list = line.split(" ")
      if len(line_list) > 1:
        words.append(line_list[0])
        labels.append(line_list[1])
      else:
        data.append([words,labels])
        words = []
        labels = []

    return data

!git clone https://github.com/kfirbar/nlp-course
train = read_data('/content/nlp-course/connl03_train.txt')
test = read_data('/content/nlp-course/connl03_test.txt')
dev = read_data('/content/nlp-course/connl03_dev.txt')


Cloning into 'nlp-course'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 58 (delta 24), reused 31 (delta 8), pack-reused 0
Unpacking objects: 100% (58/58), done.


The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [ ]:
UNK_TOKEN = 0

class Vocab:
    def __init__(self):
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
        
    def index_words(self, words):
      word_indexes = [self.index_word(w) for w in words]
      return word_indexes

    def index_tags(self, tags):
      tag_indexes = [self.tag2id[t] for t in tags]
      return tag_indexes
    
    def index_word(self, w):
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        return self.word2id[w]
            

**Task 2:** Write a function *prepare_data* that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,labels) to a pair of indexes (from Vocab). Each pair should be added to *data_sequences*, which is returned back from the function.

In [ ]:
vocab = Vocab()

def prepare_data(data, vocab):
    data_sequences = []
    # TODO - your code...
    for i in range(len(data)):
      vocab.index_words(data[i][0])
      vocab.index_tags(data[i][1])

      word_index = []
      tag_id = []
      for j in range(len(data[i][0])):
        word_index.append(vocab.word2id[data[i][0][j]])
        tag_id.append(vocab.tag2id[data[i][1][j]])
      data_sequences.append([word_index,tag_id])

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

In [ ]:
import torch.nn.functional as F

class NERNet(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, directions):
        super(NERNet, self).__init__()
        # TODO: your code...
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, bidirectional=(True if directions==2 else False))
        self.out = nn.Linear(hidden_size*directions, output_size)
        self.n_layers = n_layers
        self.directions = directions
        self.hidden_size = hidden_size
        self.output_size = output_size
    
    def forward(self, input_sentence):
        # TODO: your code...
        seq = self.embedding(input_sentence) 
        h0 = torch.zeros(self.n_layers*self.directions, len(input_sentence), self.hidden_size) 
        c0 = torch.zeros(self.n_layers*self.directions, len(input_sentence), self.hidden_size)
        seq = seq.unsqueeze(0)
        seq, _ = self.lstm(seq, (h0,c0))
        seq = seq.view(-1, seq.shape[2])
        seq= self.out(seq)  
        output = F.log_softmax(seq, dim=1) 
        return output
    
    def set_embedding(self, embs_npa):
        self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float())
        assert self.embedding.weight.shape == embs_npa.shape

**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [ ]:
def train_loop(model, n_epochs, train_sequences):
  # Loss function
  criterion = nn.CrossEntropyLoss()

  # Optimizer (ADAM is a fancy version of SGD)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  
  for e in range(1, n_epochs + 1):
    # TODO - your code goes here...
    for i in range(len(train_sequences)):
      batch_data, batch_labels  =  train_sequences[i][0] , train_sequences[i][1]
      batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)
      optimizer.zero_grad()
      output = model.forward(batch_data)
      loss = criterion(output, batch_labels)
      loss.backward()
      optimizer.step()

**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

def evaluate(model, caption, data):
  # TODO - your code goes here

  y_pred_list = []
  y_test = []

  model.eval()
  with torch.no_grad():
      for i in range(len(data)):
        batch_data, batch_labels  =  data[i][0] , data[i][1] 
        batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)
        y_test_pred = model(batch_data).to(device)
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list += y_pred_tags
        y_test += batch_labels
      model.train()
          
      y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
      y_test = [a.squeeze().tolist() for a in y_test]

      print(caption)
      print(classification_report(y_test, y_pred_list, zero_division = 1))

**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 7: (hidden_size: 800, n_layers: 1, directions: 2)

Model 8: (hidden_size: 800, n_layers: 2, directions: 2)

Model 9: (hidden_size: 800, n_layers: 3, directions: 2)

In [ ]:
# TODO - your code goes here...

models = 9
#To limit runtime we only ran on 3 epochs
n_epochs = 3
embedding_size = 300
hidden_sizes = [500]*6 + [800]*3
n_layers = [1,2,3]*3
directions = [1]*3 + 6*[2]

for i in range(models):
  model = NERNet(vocab.n_words, embedding_size, hidden_sizes[i], len(vocab.tag2id) , n_layers[i], directions[i])
  train_loop(model, n_epochs, train_sequences)
  caption = f'Test Dataset - Model {i+1} : (hidden_size: {hidden_sizes[i]}, n_layers: {n_layers[i]}, directions: {directions[i]})\n'
  evaluate(model, caption, test_sequences)
  caption = f'Dev Dataset - Model {i+1} : (hidden_size: {hidden_sizes[i]}, n_layers: {n_layers[i]}, directions: {directions[i]})\n'
  evaluate(model, caption, dev_sequences)

Test Dataset - Model 1 : (hidden_size: 500, n_layers: 1, directions: 1)

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      6567
           1       0.61      0.23      0.34       434
           2       0.45      0.07      0.12       296
           3       0.76      0.52      0.62       343
           4       0.86      0.34      0.49        53
           5       0.35      0.29      0.32       350
           6       0.67      0.14      0.24       200

    accuracy                           0.83      8243
   macro avg       0.65      0.37      0.43      8243
weighted avg       0.80      0.83      0.80      8243

Dev Dataset - Model 1 : (hidden_size: 500, n_layers: 1, directions: 1)

              precision    recall  f1-score   support

           0       0.86      0.98      0.91      3096
           1       0.59      0.22      0.32       200
           2       0.89      0.10      0.18       157
           3       0.79      0.54      0.

**Task 7:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

In [ ]:
# TODO - your code goes here...
import numpy as np

def read_glove(path):
  vocab,embeddings = [],[]
  with open(path,'rt') as fi:
      full_content = fi.read().strip().split('\n')
  for i in range(len(full_content)):
      i_word = full_content[i].split(' ')[0]
      i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
      vocab.append(i_word)
      embeddings.append(i_embeddings)

  vocab_npa = np.array(vocab)
  embs_npa = np.array(embeddings)

  vocab_npa = np.insert(vocab_npa, 0, "__unk__")
  unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)  # Initializing 'unknown' word embedding to mean
  embs_npa = np.vstack((unk_emb_npa,embs_npa))
  return vocab_npa, embs_npa

In [ ]:
#Download glove zip then run unzip
import os
if not os.path.exists('glove'):
  !wget http://nlp.stanford.edu/data/glove.6B.zip && unzip glove.6B.zip && mkdir glove && mv glove*.txt glove

GLOVE_FILENAME = 'glove/glove.6B.300d.txt'

--2021-06-08 19:50:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-08 19:50:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-08 19:50:01--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# repeat task 6

vocab_npa, embs_npa = read_glove(GLOVE_FILENAME)

for i in range(models):
  model = NERNet(vocab.n_words, embedding_size, hidden_sizes[i], len(vocab.tag2id), n_layers[i], directions[i])
  model.set_embedding(embs_npa)
  train_loop(model, n_epochs, train_sequences)
  caption = f'Test Dataset - Model {i+1} : (hidden_size: {hidden_sizes[i]}, n_layers: {n_layers[i]}, directions: {directions[i]})\n'
  evaluate(model, caption, test_sequences)
  caption = f'Dev Dataset - Model {i+1} : (hidden_size: {hidden_sizes[i]}, n_layers: {n_layers[i]}, directions: {directions[i]})\n'
  evaluate(model, caption, dev_sequences)

Test Dataset - Model 1 : (hidden_size: 500, n_layers: 1, directions: 1)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89      6567
           1       0.32      0.05      0.09       434
           2       0.00      0.00      0.00       296
           3       0.49      0.24      0.32       343
           4       0.55      0.11      0.19        53
           5       0.12      0.06      0.08       350
           6       0.67      0.04      0.08       200

    accuracy                           0.79      8243
   macro avg       0.42      0.21      0.23      8243
weighted avg       0.71      0.79      0.73      8243

Dev Dataset - Model 1 : (hidden_size: 500, n_layers: 1, directions: 1)

              precision    recall  f1-score   support

           0       0.80      0.97      0.88      3096
           1       0.37      0.05      0.09       200
           2       0.00      0.00      0.00       157
           3       0.45      0.16      0.

**Good luck!**